In [355]:
from helpers import cd_to_datetime, datetime_to_str

class NearEarthObject:
    """A near-Earth object (NEO).

    An NEO encapsulates semantic and physical parameters about the object, such
    as its primary designation (required, unique), IAU name (optional), diameter
    in kilometers (optional - sometimes unknown), and whether it's marked as
    potentially hazardous to Earth.

    A `NearEarthObject` also maintains a collection of its close approaches -
    initialized to an empty collection, but eventually populated in the
    `NEODatabase` constructor.
    """

    def __init__(self, designation, name=None, diameter=float('nan'), hazardous='N'):
        """Create a new `NearEarthObject`.
        
        :param info: 
        required unique designation (str), 
        optional name (str), diameter (float), hazardous (bool)
        """
        
        self.designation = str(designation)
        self.name = str(name).title()
        self.diameter = diameter
        self.hazardous = hazardous
        
        self.approaches = []

    @property
    def fullname(self):
        """Return a representation of the full name of this NEO."""
        
        if self.name is None:
            return self.designation      
        else:
            return self.designation + " (" + self.name + ")"
    
    @property
    def hazard(self):
        """Return 'is/is not' depending on boolean value of hazardous."""
        
        if self.hazardous == 'N':
            hazard = 'is not'
        else:
            hazard = 'is'
        return hazard

    def __str__(self):
        """Return `str(self)`."""
        
        formatted_string = "NEO {} has a diameter {} km and {} potentially hazardous.".format(
        self.fullname, self.diameter, self.hazard
        )

        return formatted_string
  
    
    def __hash__(self):
        return hash(str(self))
    
    def __eq__(self, neo):
        return self.designation is neo.designation

class CloseApproach:
    """A close approach to Earth by an NEO.

    A `CloseApproach` encapsulates information about the NEO's close approach to
    Earth, such as the date and time (in UTC) of closest approach, the nominal
    approach distance in astronomical units, and the relative approach velocity
    in kilometers per second.

    A `CloseApproach` also maintains a reference to its `NearEarthObject` -
    initally, this information (the NEO's primary designation) is saved in a
    private attribute, but the referenced NEO is eventually replaced in the
    `NEODatabase` constructor.
    """

    def __init__(self, neo, time, distance=float('nan'), velocity=float('nan')):
        """Create a new `CloseApproach`.

        :param info: 
        required unique designation (str), can be passed from the neo type'
        required date (str);
        optional: distance (float), velocity (bool), neo (neo).
        """

        
        self.neo = neo
        self._designation = neo.designation
        self.time = cd_to_datetime(time)
        
        self.distance = distance
        self.velocity = velocity

        
    @property
    def time_str(self):
        """Return a formatted representation of this `CloseApproach`'s approach time.

        The value in `self.time` should be a Python `datetime` object. While a
        `datetime` object has a string representation, the default representation
        includes seconds - significant figures that don't exist in our input
        data set.

        The `datetime_to_str` method converts a `datetime` object to a
        formatted string that can be used in human-readable representations and
        in serialization to CSV and JSON files.
        """

        return datetime_to_str(self.time)

    
    def __str__(self):
        """Return `str(self)`."""
        
        formatted_string = "- On {} NEO {} approaching with speed {} km/s and distance {} au.".format(
        self.time_str, self.neo.fullname, self.velocity, self.distance
        )
            
        return formatted_string

    def __repr__(self):
        """Return `repr(self)`, a computer-readable string representation of this object."""
        
        formatted_string = "CloseApproach(time={}, distance={}, velocity={}, neo={})".format(
        self.time_str, self.distance, self.velocity, self.neo)
        
        return formatted_string

In [343]:
neo = NearEarthObject(123, 'Barsik', 21.21, True)
cap = CloseApproach(neo, '1900-Dec-27 12:12', 10, 0.1)
neo.approaches.append(cap)
new_dic = {neo:cap}
cap.time_str

'1900-12-27 12:12'

In [344]:
print(neo)
print(cap)
neo.approaches
new_dic

NEO 123 (Barsik) has a diameter 21.21 km and is potentially hazardous.
- On 1900-12-27 12:12 NEO 123 (Barsik) approaching with speed 0.1 km/s and distance 10 au.


{<__main__.NearEarthObject at 0x7fecff260bb0>: CloseApproach(time=1900-12-27 12:12, distance=10.00, velocity=0.10, neo=<__main__.NearEarthObject object at 0x7fecff260bb0>)}

### Loading datasets with `extract.py`

In [512]:
"""Extract data on near-Earth objects and close approaches from CSV and JSON files.

The `load_neos` function extracts NEO data from a CSV file, formatted as
described in the project instructions, into a collection of `NearEarthObject`s.

The `load_approaches` function extracts close approach data from a JSON file,
formatted as described in the project instructions, into a collection of
`CloseApproach` objects.

The main module calls these functions with the arguments provided at the command
line, and uses the resulting collections to build an `NEODatabase`.

You'll edit this file in Task 2.
"""
import csv
import json

#from models import NearEarthObject, CloseApproach


def load_neos(neo_csv_path = 'data/neos.csv') -> dict:
    """Read near-Earth object information from a CSV file.

    :param neo_csv_path: A path to a CSV file containing data about near-Earth objects.
    :return: A collection of `NearEarthObject`s.
    """

    with open(neo_csv_path, 'r') as file:
        neo_data = [row for row in csv.DictReader(file)]
        
    neos_dict = {}
    for i in range(len(neo_data)):
        neos_dict[neo_data[i]['pdes']] = NearEarthObject(
            neo_data[i]['pdes'], 
            neo_data[i]['name'], 
            neo_data[i]['diameter'], 
            neo_data[i]['pha'])
    
    return neos_dict



def load_approaches(cad_json_path = 'data/cad.json') -> dict:
    """Read close approach data from a JSON file.

    :param neo_csv_path: A path to a JSON file containing data about close approaches.
    :return: A collection of `CloseApproach`es.
    """
    with open(cad_json_path, 'r') as file:
        cads_dict = json.load(file)
    
    return cads_dict

### Encapsulate the data in a `NEODatabase`

In [565]:
neos = load_neos()
approaches = load_approaches()


In [567]:
approaches['fields']

['des',
 'orbit_id',
 'jd',
 'cd',
 'dist',
 'dist_min',
 'dist_max',
 'v_rel',
 'v_inf',
 't_sigma_f',
 'h']

In [566]:
len(neos)

23967

In [685]:
"""A database encapsulating collections of near-Earth objects and their close approaches.
    
    A `NEODatabase` holds an interconnected data set of NEOs and close approaches.
    It provides methods to fetch an NEO by primary designation or by name, as well
    as a method to query the set of close approaches that match a collection of
    user-specified criteria.
    
    Under normal circumstances, the main module creates one NEODatabase from the
    data on NEOs and close approaches extracted by `extract.load_neos` and
    `extract.load_approaches`.
    
    """
#from extract import load_neos, load_approaches

class NEODatabase:
    """A database of near-Earth objects and their close approaches.
        
        A `NEODatabase` contains a collection of NEOs and a collection of close
        approaches. It additionally maintains a few auxiliary data structures to
        help fetch NEOs by primary designation or by name and to help speed up
        querying for close approaches that match criteria.
        """
    def __init__(self, neos, approaches):
        """Create a new `NEODatabase`.
            
            As a precondition, this constructor assumes that the collections of NEOs
            and close approaches haven't yet been linked - that is, the
            `.approaches` attribute of each `NearEarthObject` resolves to an empty
            collection, and the `.neo` attribute of each `CloseApproach` is None.
            
            However, each `CloseApproach` has an attribute (`._designation`) that
            matches the `.designation` attribute of the corresponding NEO. This
            constructor modifies the supplied NEOs and close approaches to link them
            together - after it's done, the `.approaches` attribute of each NEO has
            a collection of that NEO's close approaches, and the `.neo` attribute of
            each close approach references the appropriate NEO.
            
            :param neos: A collection of `NearEarthObject`s.
            :param approaches: A collection of `CloseApproach`es.
            """
        self.neos = neos
        self.approaches = approaches
        
        for pdes, neo in self.neos.items():
            for des, approach in self.approaches.items():
                if pdes == des:
                    self.neos[pdes].approaches.append(approach)
                    self.approaches[pdes].neo = neo
    


    def get_neo_by_designation(self, designation):
        """Find and return an NEO by its primary designation.

            If no match is found, return `None` instead.

            Each NEO in the data set has a unique primary designation, as a string.

            The matching is exact - check for spelling and capitalization if no
            match is found.

            :param designation: The primary designation of the NEO to search for.
            :return: The `NearEarthObject` with the desired primary designation, or `None`.
            """
        if self.neos[designation]:
            print (self.neos[designation])
        else:
            print("No NEO designation {} was found.".format(self._neos[designation]))
        return
                    


    def get_neo_by_name(self, name):
        """Find and return an NEO by its name.

            If no match is found, return `None` instead.

            Not every NEO in the data set has a name. No NEOs are associated with
            the empty string nor with the `None` singleton.

            The matching is exact - check for spelling and capitalization if no
            match is found.

            :param name: The name, as a string, of the NEO to search for.
            :return: The `NearEarthObject` with the desired name, or `None`.
            """
        for neo in neos.values():
            if name == neo.name:
                print(neo)
            else:
                print("No NEO name {} was found.".format(neo.name))
        return

    def query(self, filters=()):
        """Query close approaches to generate those that match a collection of filters.

            This generates a stream of `CloseApproach` objects that match all of the
            provided filters.

            If no arguments are provided, generate all known close approaches.

            The `CloseApproach` objects are generated in internal order, which isn't
            guaranteed to be sorted meaninfully, although is often sorted by time.

            :param filters: A collection of filters capturing user-specified criteria.
            :return: A stream of matching `CloseApproach` objects.
            """

            # TODO: Generate `CloseApproach` objects that match all of the filters.
        for approach in self.approaches:
            yield approach


In [686]:
neo_db = NEODatabase(neos, approaches)

In [687]:
print(neo_db.neos['433'].approaches)

[]


In [688]:
neo_db.get_neo_by_name('Eros')._approaches

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.
No NEO name Albert was found.
No NEO name Alinda was found.
No NEO name Ganymed was found.
No NEO name Amor was found.
No NEO name Icarus was found.
No NEO name Betulia was found.
No NEO name Geographos was found.
No NEO name Ivar was found.
No NEO name Toro was found.
No NEO name Apollo was found.
No NEO name Antinous was found.
No NEO name Daedalus was found.
No NEO name Cerberus was found.
No NEO name Sisyphus was found.
No NEO name Quetzalcoatl was found.
No NEO name Boreas was found.
No NEO name Cuyo was found.
No NEO name Anteros was found.
No NEO name Tezcatlipoca was found.
No NEO name Midas was found.
No NEO name Baboquivari was found.
No NEO name Anza was found.
No NEO name Aten was found.
No NEO name Bacchus was found.
No NEO name Ra-Shalom was found.
No NEO name Adonis was found.
No NEO name Tantalus was found.
No NEO name Aristaeus was found.
No NEO name Oljato was found.
No NEO name Pele was found.
N

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

...

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was found.
No NEO name  was

AttributeError: 'NoneType' object has no attribute '_approaches'

In [690]:
neo_db.get_neo_by_designation('433')

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.


### Example of class methods to dynamically create new classes:

In [611]:
class Dog():

    def __init__(self, name:str, age:int, breed:str, weight:int):
        """Create a new dog"""
        self.breed = breed
        self.weight = weight
        self.name = name
        self.age = age
        
    def __gt__(self, other):
        return self.age > other.age
    
    def __str__(self):
        return str(self.name)
    
    def speak(self) -> None:
        """Make the dog bark"""
        print(f'{self.name} says, "woof"')
    @classmethod    
    def spawn(cls, name, mother, father):
        breed = mother.breed
        if mother.breed != father.breed:
            breed = f'{mother.breed}-{father.breed} Mix'
        weight = (mother.weight + father.weight) / (2 * 10)
        return cls(name, 0, breed, weight)

if __name__ == "__main__":
    sally = Dog('Sally', 6, 'chihuahua', 7)
    henry = Dog('Henry', 7, 'terrier', 15)
    trixy = Dog.spawn('Trixy', sally, henry)
    print(trixy.breed)

chihuahua-terrier Mix


In [640]:
class Customer():

    def __init__(self, name, family, tier=('free', 0)):
        self.name = name
        self.family = family
        self.tier = tier[0]
        self.cost = tier[1]


    @property
    def full_name(self):
        return f"{self.name} {self.family}"


    def can_access(self, book_info):
        return book_info['tier'] in self.tier


    @classmethod
    def premium(cls, name, family):
        return cls(name, family, tier=('premium', 10))


    def bill_for(self, months):
        return self.cost*months

In [638]:
c = Customer("bob", 'smith')

In [646]:
c.bill_for(2)

0

In [632]:
class C():

    def __init__(self, name:str, family:str, tier=('free', 0)):
        self.name = name
        self.family = family
        self.tier = tier[0]
        self.cost = tier[1]
    
    @property
    def full_name(self):
        full_name
        return ({}{}".format(self.name, self.family)
        

SyntaxError: invalid syntax (<ipython-input-632-930b5eebdd95>, line 11)

In [631]:
cc = C('bob', 'cmith')

AttributeError: can't set attribute

In [692]:
(19-3.5)*0.4

6.2

In [623]:
cc.family

'cmith'

In [624]:
cc.tier

'free'

In [625]:
cc.cost

0